Copyright 2021 The TensorFlow Authors.
```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Tiền xử lý văn bản với BERT

## Tổng quan

Tiền xử lý văn bản là quá trình chuyển đổi từ đầu-đến-cuối của văn bản thô thành đầu vào số nguyên của mô hình. Các mô hình xử lý ngôn ngữ tự nhiên (NLP) thường đi kèm với hàng trăm (nếu không phải hàng nghìn) dòng mã Python để tiền xử lý văn bản. Tiền xử lý văn bản thường là một thách thức cho các mô hình bởi vì:
* **Độ nghiêng phục vụ-huấn luyện**: Ngày càng khó khăn để đảm bảo rằng logic tiền xử lý của các đầu vào của mô hình là nhất quán ở tất cả các giai đoạn phát triển mô hình (chẳng hạn tiền huấn luyện, tinh chỉnh, đánh giá và suy luận). Sử dụng các siêu tham số khác nhau, token hoá, các thuật toán tiền xử lý chuỗi hoặc đơn giản là đóng gói các đầu vào mô hình không nhất quán ở các giai đoạn khác nhau có thể mang lại lỗi khó gỡ và các tác động không tốt cho mô hình.
* **Hiệu quả và linh hoạt**: Trong khi tiền xử lý có thể làm ở ngoại tuyến (chẳng hạn bằng cách viết các đầu ra đã được xử lý vào tập tin lưu trên đĩa và sau đó xem xét lại các dữ liệu đã được tiền xử lý trong đường tin đầu vào), phương pháp này phát sinh thêm chi phí đọc-ghi dữ liệu. Tiền xử lý ngoại tuyến thường bất tiện nếu có các quyết định tiền xử lý cần xảy ra động. Thử nghiệm với một tuỳ chọn khác nhau sẽ yêu cầu tạo lại tập dữ liệu một lần nữa.
* **Giao diện mô hình phức tạp**: Các mô hình văn bản dễ hiểu hơn nhiều khi các đầu vào của nó là các văn bản thuần. Nó cũng gây khó hiểu một mô hình khi đầu vào yêu cầu các bước mã hoá gián tiếp bổ sung. Giảm độ phức tạp của quá trình tiền xử lý được đánh giá cao cho quá trình sửa lỗi mô hình, phục vụ và đánh giá.

Thêm vào đó, giao diện mô hình đơn giản hơn cũng tạo thêm sự thuận tiện để thử mô hình (chẳng hạn dùng để suy luận hay huấn luyện) trên các tập dữ liệu khác nhau, chưa được khám phá.

## Tiền xử lý dữ liệu với TF.Text

Sử dụng các API tiền xử lý văn bản của TF.Text, chúng ta có thể xây dựng một hàm tiền xử lý mà có thể chuyển đổi một tập dữ liệu văn bản của người dùng thành các đầu vào số nguyên của mô hình. Người dùng có thể đóng gói trực tiếp tiền xử lý như là một phần của mô hình để giảm bớt các vấn đề đã đề cập ở trên.

Hướng dẫn này sẽ trình bày cách sử dụng các hoạt động tiền xử lý TF.Text để chuyển đổi dữ liệu văn bản thành các đầu vào cho mô hình BERT và các đầu vào cho nhiệm vụ tiền huấn luyện mặt nạ ngôn ngữ được mô tả trong "*Masked LM and Masking Procedure*" của [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf). Quá trình liên quan đến token hoá văn bản thành các đơn vị từ con, cắt xen nội dung để vừa khít kích thước và trích xuất nhãn cho nhiệm vụ mô hình hoá mặt nạ ngôn ngữ.

### Văn bản

Nhập các gói và thư viện cần thiết:

In [ ]:
!pip install -q -U tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_text as text
import functools

Dữ liệu của chúng ta chứa hai đặc trưng văn bản và chúng ta có thể tạo một ví dụ `tf.data.Dataset`. Mục tiêu của chúng ta là tạo một hàm mà chúng ta có thể cung cấp `Dataset.map()` vào huấn luyện mô hình.

In [ ]:
examples = {
    "text_a": [
      b"Sponge bob Squarepants is an Avenger",
      b"Marvel Avengers"
    ],
    "text_b": [
     b"Barack Obama is the President.",
     b"President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))

### Toeken hoá

Bước đầu tiên là chạy tiền xử lý chuỗi bất kỳ và token tập dữ liệu. Điều này có thể thực hiện bởi `text.BertTokenizer`, nó là một hàm `text.Splitter` token các câu thành các từ con hoặc các mảnh từ cho [mô hình BERT](https://github.com/google-research/bert) với một từ vựng được tạo ra từ [thuật toán Wordpiece](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_the_algorithm).

Từ vựng có thể là các điểm kiểm tra BERT được tạo trước đó, hoặc bạn có thể tạo một từ mới trên dữ liệu của bạn. Đối với các mục đích của ví dụ này, hãy tạo một từ vựng đồ chơi.

In [ ]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5
 
_VOCAB_SIZE = len(_VOCAB)

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=_VOCAB,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
      value_dtype=tf.int64),
      num_oov_buckets=1
)

Xây dựng một `text.BertTokenizer` sử dụng từ vựng ở trên và token đầu vào văn bản thành một `RaggedTensor`.

In [ ]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.string)
bert_tokenizer.tokenize(examples["text_a"])

In [ ]:
bert_tokenizer.tokenize(examples["text_b"])

Đầu ra văn bản từ `text.BertTokenizer` cho phép chúng ta xem cách văn bản được token hoá, nhưng mô hình yêu câu các ID số nguyên. Chúng ta có thể đặt tham số `token_out_type` thành `tf.int64` để lấy các ID nguyên (là các chỉ số trong từ vựng).

In [ ]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.int64)
segment_a = bert_tokenizer.tokenize(examples["text_a"])
segment_a

In [ ]:
segment_b = bert_tokenizer.tokenize(examples["text_b"])
segment_b

`text.BertTokenizer` trả về một RaggedTensor với hình dạng `[batch, num_tokens, num_wordpieces]`. Bởi vì chúng ta không cần chiều `num_tokens` bổ sung cho trường hợp sử dụng hiện tại, chúng ta có thể gộp hai chiều cuối cùng để có được một `RaggedTensor` với hình dạng `[batch, num_wordpieces]`:

In [ ]:
segment_a = segment_a.merge_dims(-2, -1)
segment_a

In [ ]:
segment_b = segment_b.merge_dims(-2, -1)
segment_b

### Cắt xén Nội dung

Đầu vào chính của BERT là ghép nối hai câu. Tuy nhiên, BERT yêu cầu các đầu vào phải có kích thước và hình dạng cố định và chúng ta có thể có nội dung vượt quá ngân sách của chúng ta.

Chúng ta có thể giải quyết vấn đề này bằng cách sử dụng một `text.Trimmer` để cắt nội dung theo kích thước đã xác định trước (một khi được nối theo chiều dọc cuối cùng). Có một vài kiểu `text.Trimmer` khác nhau mà chọn nội dung để giữ lại bằng các thuật toán khác nhau. `text.RoundRobinTrimmer` là một ví dụ sẽ phân bổ hạn ngạch như nhau cho mỗi đoạn nhưng có thể cắt xét phần cuối của các câu. `text.WaterfallTrimmer` sẽ cắt xén bắt đầu từ cuối của câu cuối cùng.

Trong ví dụ của chúng ta, chúng ta sẽ sử dụng `RoundRobinTrimmer` mà chọn các mục từ mỗi đoạn từ trái-sang-phải.

In [ ]:
trimmer = text.RoundRobinTrimmer(max_seq_length=[_MAX_SEQ_LEN])
trimmed = trimmer.trim([segment_a, segment_b])
trimmed

`trimmed` bây giờ có chứa các đoạn mà số lượng các phần tử trong mỗi lô là 8 phần tử (khi được nối dọc theo trục axis=-1).

### Nối đoạn

Bây giờ chúng ta đã có các đoạn, chúng ta có thể nối chúng vào nhau để có một `RaggedTensor` duy nhất. BERT sử dụng các token đặc biệt để chỉ ra phần đầu (`[CLS]`) và phần cuối của một đoạn (`[SEP]`). Chúng ta cũng cần một `RaggedTensor` cho biết mục nào trong `Tensor` được nối thuộc về đoạn nào. Chúng ta có thể sử dụng `text.combine_segments()` để lấy cả `Tensor` với token đặc biệt được chèn.

In [ ]:
segments_combined, segments_ids = text.combine_segments(
  [segment_a, segment_b],
  start_of_sequence_id=_START_TOKEN, end_of_segment_id=_END_TOKEN)
segments_combined, segments_ids

### Nhiệm vụ Mô hình Mặt nạ Ngôn ngữ

Bây giờ chúng ta có các đầu vao cơ bản và có thể bắt đầu trích xuất các đặc trưng cần thiết cho nhiệm vụ "*Masked LM and Masking Procedure*" được mô tả trong [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf).

Nhiệm vụ mô hình mặt nạ ngôn ngữ có hai vấn đề phụ cho chúng ta để suy nghĩ là: (1) những mục nào để chọn cho mặt nạ và (2) chúng được gán giá trị nào?

#### Lựa chọn mục

Bởi vì chúng ta sẽ chọn các mục ngẫu nhiên cho mặt nạ, chúng ta sẽ sử dụng một `text.RandomItemSelector`. `RandomItemSelector` ngẫu nhiên chọn các mục trong một lô chủ đè để hạn chế `(max_selections_per_batch, selection_rate và unselectable_ids)` và trả về mọt mặt nạ boolean cho biết các mục nào được chọn.

In [ ]:
random_selector = text.RandomItemSelector(
    max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH,
    selection_rate=0.2,
    unselectable_ids=[_START_TOKEN, _END_TOKEN, _UNK_TOKEN]
)
selected = random_selector.get_selection_mask(
    segments_combined, axis=1)
selected

#### Chọn giá trị được che giấu

Phương pháp này được mô tả trong bài báo BERT nguyên bản để chọn giá trị cho mặt nạ:

Đối với `mask_token_rate` của thời gian, thay thế mục này với token `[MASK]`:

```
"my dog is hairy" -> "my dog is [MASK]"
```

Đối với `mask_token_rate` của thời gian, thay thế mục này với một từ ngẫu nhiên:

```
"my dog is hairy" -> "my dog is apple"
```

Đối với `1 - mask_token_rate - random_token_rate` của thời gian, giữ mục không bị thay đổi:

```
"my dog is hairy" -> "my dog is hairy."
```

`text.MaskedValuesChooser` đóng gói logic này và có thể được dùng cho hàm tiền xử lý dữ liệu của chúng ta. Dưới đây là một ví dụ của `MaskValuesChooser` trả về một `mask_token_rate` của 80% và `random_token_rate` mặc định:

In [ ]:
input_ids = tf.ragged.constant([[19, 7, 21, 20, 9, 8], [13, 4, 16, 5], [15, 10, 12, 11, 6]])
mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, 0.8)
mask_values_chooser.get_mask_values(input_ids)

Khi được cung cấp một đầu vào `RaggedTensor`, `text.MaskValuesChooser` trả về một `RaggedTensor` của cùng hình dạng với hoặc `_MASK_VALUE (0)`, một ID ngẫu nhien hoặc cùng ID không bị thay đổi.

#### Tạo các đầu vào cho Nhiệm vụ Mô hình Mặt nạ Ngôn ngữ

Bây giờ chúng ta có một `RandomItemSelector` để giúp chúng ta lựa chọn các mục cho mặt nạ và `text.MaskValuesChooser` để gán các giá trị, chúng ta có thể sử dụng `text.mask_language_model()` để lắp tất cả các đầu vào của nhiệm vụ này cho mô hình BERT của chúng ta.

In [ ]:
masked_token_ids, masked_pos, masked_lm_ids = text.mask_language_model(
  segments_combined,
  item_selector=random_selector, mask_values_chooser=mask_values_chooser

Cùng đi sâu và thám hiểm các đầu ra của `mask_language_model()`. Đầu ra của `masked_token_ids` là:

In [ ]:
masked_token_ids

Nhớ rằng đầu vào của chúng ta được mã hoá sử dụng một từ vựng. Nếu chúng ta giải mã `masked_token_ids` sử dụng từ vựng của chúng ta, chúng ta có:

In [ ]:
tf.gather(_VOCAB, masked_token_ids)

Chú ý rằng một vài token mảnh từ đã được thay thế với `[MASK]`, `[RANDOM]` hoặc một giá trị ID khác. Đầu ra `masked_pos` cho chúng ta các chỉ số (trong lô tương ứng) của các token đã được thay thế.

In [ ]:
masked_pos

`masked_lm_ids` cho chúng ta giá trị ban đầu của token.

In [ ]:
masked_lm_ids

Chúng ta có thể giải mã một lần nữa các ID ở đây để có các giá trị dễ đọc.

In [ ]:
tf.gather(_VOCAB, masked_lm_ids)

### Đầu vào Mô hình Đệm

Bây giờ chúng ta đã có tất cả các đầu vào cho mô hình của mình, bước cuối cùng trong quá trình tiền xử lý là đóng gói chúng thành các `Tensor` 2 chiều cố định với đệm và cũng tạo ra một `Tensor` mặt nạ cho biết các giá trị là giá trị `pad`. Chúng ta có thể sử dụng `text.pad_model_inputs()` để giúp chúng ta thực hiện nhiệm vụ này.

In [ ]:
# Prepare and pad combined segment inputs
input_word_ids, input_mask = text.pad_model_inputs(
  masked_token_ids, max_seq_length=_MAX_SEQ_LEN)
input_type_ids, _ = text.pad_model_inputs(
  masked_token_ids, max_seq_length=_MAX_SEQ_LEN)

# Prepare and pad masking task inputs
masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
  masked_token_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
masked_lm_ids, _ = text.pad_model_inputs(
  masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

model_inputs = {
    "input_word_ids": input_word_ids,
    "input_mask": input_mask,
    "input_type_ids": input_type_ids,
    "masked_lm_ids": masked_lm_ids,
    "masked_lm_positions": masked_lm_positions,
    "masked_lm_weights": masked_lm_weights,
}
model_inputs

## Xem lại

Hãy xem lại những gì chúng ta có cho đến nay và lắp chức năng tiền xử lý của chúng ta. Đây là những gì chúng ta có:

In [ ]:
def bert_pretrain_preprocess(vocab_table, features):
  # Input is a string Tensor of documents, shape [batch, 1].
  text_a = features["text_a"]
  text_b = features["text_b"]

  # Tokenize segments to shape [num_sentences, (num_words)] each.
  tokenizer = text.BertTokenizer(
      vocab_table,
      token_out_type=tf.int64)
  segments = [tokenizer.tokenize(text).merge_dims(
      1, -1) for text in (text_a, text_b)]

  # Truncate inputs to a maximum length.
  trimmer = text.RoundRobinTrimmer(max_seq_length=6)
  trimmed_segments = trimmer.trim(segments)

  # Combine segments, get segment ids and add special tokens.
  segments_combined, segment_ids = text.combine_segments(
      trimmed_segments,
      start_of_sequence_id=_START_TOKEN,
      end_of_segment_id=_END_TOKEN)

  # Apply dynamic masking task.
  masked_input_ids, masked_lm_positions, masked_lm_ids = (
      text.mask_language_model(
        segments_combined,
        random_selector,
        mask_values_chooser,
      )
  )
  
  # Prepare and pad combined segment inputs
  input_word_ids, input_mask = text.pad_model_inputs(
    masked_token_ids, max_seq_length=_MAX_SEQ_LEN)
  input_type_ids, _ = text.pad_model_inputs(
    masked_token_ids, max_seq_length=_MAX_SEQ_LEN)

  # Prepare and pad masking task inputs
  masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
    masked_token_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
  masked_lm_ids, _ = text.pad_model_inputs(
    masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

  model_inputs = {
      "input_word_ids": input_word_ids,
      "input_mask": input_mask,
      "input_type_ids": input_type_ids,
      "masked_lm_ids": masked_lm_ids,
      "masked_lm_positions": masked_lm_positions,
      "masked_lm_weights": masked_lm_weights,
  }
  return model_inputs

Chúng ta đã xây dựng trước đó một `tf.data.Dataset` và chúng ta có thể sử dụng ngay hàm tiền xử lý đã lắp vào `bert_pretrain_preprocess()` trong `Dataset.map()`. Điều này cho phép chúng ta tạo một đường tin đầu vào cho chuyển đổi chuỗi thô thành các đầu vào số nguyên và cho vào trực tiếp mô hình của chúng ta.

In [ ]:
dataset = tf.data.Dataset.from_tensors(examples)
dataset = dataset.map(functools.partial(
    bert_pretrain_preprocess, lookup_table))

next(iter(dataset))